In [1]:
import pandas as pd
import numpy as np
from hyperopt import fmin, tpe, hp
from sklearn.model_selection import train_test_split

from ipynb.fs.full.features import features_independientes_precio, features_dependientes_precio

df_train = pd.read_csv('./data/train.csv')

# Para usarse con el submit a Kaggle
df_test = pd.read_csv('./data/test.csv')

In [10]:
def RMSLE(actual, pred):
    return (np.mean((np.log(actual + 1) - np.log(pred + 1)) ** 2)) **.5

In [20]:
def llenar_nulls(df):
    df['metrostotales'] = df['metrostotales'].fillna(df['metroscubiertos'])
    df['metroscubiertos'] = df['metrostotales'].fillna(df['metrostotales'])
    
    df['habitaciones'] = df['habitaciones'].fillna(df['habitaciones'].mean())
    df['garages'] = df['garages'].fillna(df['garages'].mean())
    df['banos'] = df['banos'].fillna(df['banos'].mean())

df_train_f = features_independientes_precio(df_train)
df_train_f = features_dependientes_precio(df_train_f, df_train)

df_test_f = features_independientes_precio(df_test)
df_test_f = features_dependientes_precio(df_test_f, df_train)

llenar_nulls(df_train_f)
llenar_nulls(df_test_f)

In [27]:
from sklearn.neural_network import MLPRegressor


df_nn = df_train_f.copy().drop(['titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad', 'provincia',
                                'fecha', 'zona', 'intervalo_metros_totales', 'intervalo_metros_cubiertos', 'id', 
                               'antiguedad', 'idzona', 'lat', 'lng', 'porcentaje_metros',
                               'promedio_metros_tipo_propiedad'], axis=1)

df_nn = df_nn

df_nn_test, df_nn_train = train_test_split(df_nn, test_size=0.25, random_state=1)

y_test = df_nn_test['precio']
y_train = df_nn_train['precio']
x_test = df_nn_test.drop('precio', axis=1)
x_train = df_nn_train.drop('precio', axis=1)

mlp = MLPRegressor(hidden_layer_sizes=(100, 100, 100, 100, 100), verbose=True)
mlp.fit(x_train,y_train)

mlpr_rmsle_train = RMSLE(y_train, mlp.predict(x_train))
mlpr_rmsle = RMSLE(y_test, mlp.predict(x_test))
print(f"RMSLE MLPRegressor (train): {mlpr_rmsle_train:.5f}")
print(f"RMSLE MLPRegressor: {mlpr_rmsle:.5f}")

Iteration 1, loss = 1188951620156.67285156
Iteration 2, loss = 1048615927692.77355957
Iteration 3, loss = 1049346973713.01477051
Iteration 4, loss = 1047711480503.58972168
Iteration 5, loss = 1054650397454.03149414
Iteration 6, loss = 1049987161003.58410645
Iteration 7, loss = 1045345552526.09814453
Iteration 8, loss = 1046690773430.28381348
Iteration 9, loss = 1047746500249.24475098
Iteration 10, loss = 1045041636071.42968750
Iteration 11, loss = 1045859739170.29858398
Iteration 12, loss = 1044652713465.47009277
Iteration 13, loss = 1045979201350.86291504
Iteration 14, loss = 1044721701273.66284180
Iteration 15, loss = 1045992478841.58276367
Iteration 16, loss = 1042409270639.68310547
Iteration 17, loss = 1042253195948.07250977
Iteration 18, loss = 1040890062745.46447754
Iteration 19, loss = 1042117016128.13647461
Iteration 20, loss = 1043290229029.99108887
Iteration 21, loss = 1044072917983.40368652
Iteration 22, loss = 1044911899226.28808594
Iteration 23, loss = 1042091236392.143920